In [ ]:
# Import required libraries  
import os  
from openai import OpenAI
from dotenv import load_dotenv  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient 
from azure.search.documents.models import (
    VectorizedQuery,
    VectorFilterMode,    
)
  
# Configure environment variables  
load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
model: str = "text-embedding-ada-002" 
credential = AzureKeyCredential(key)
openai_client = OpenAI()
openai_model = os.getenv("OPENAI_MODEL")

In [ ]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid

# method to get the token length with the encoding
tokenizer_name = tiktoken.encoding_for_model("gpt-4-1106-preview")
tokenizer = tiktoken.get_encoding(tokenizer_name.name)

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=8000, # this depends on which model you might use, for example with the 16k GPT models setting this to 8k is reasonable and maybe higher
    chunk_overlap=100,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""],
)

# get a UUID - URL safe, Base64
def get_a_uuid():
    return str(uuid.uuid4())

#function to return the number of tokens in a string
def num_tokens_from_string(string: str, model_name: str) -> int:
    """Returns the number of tokens in a text string."""
    #encoding = tiktoken.get_encoding(encoding_name)
    encoding = tiktoken.encoding_for_model(model_name)
    token_integers = encoding.encode(string)
    num_tokens = len(token_integers)

    return num_tokens

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai_client.embeddings.create(input = [text], model=model).data[0].embedding

def open_file(filepath):
        with open(filepath, "r", encoding="utf-8", errors="ignore") as infile:
            return infile.read()

In [ ]:
from rich.markdown import Markdown

def make_spr_by_file(filepath):        
    messages = [ 
                {
                "role": "system",
                "content": open_file("./system_spr.md")
                },
                {
                "role": "user",
                "content": open_file(filepath)
                }
                ]

    chat_completion = openai_client.chat.completions.create(
                        model=openai_model,
                        messages=messages,
                    )

    packed_answer = chat_completion.choices[0].message.content
    
    #return Markdown(packed_answer)
    return (packed_answer)

def make_spr_by_content(content):        
    messages = [ 
                {
                "role": "system",
                "content": open_file("./system_spr.md")
                },
                {
                "role": "user",
                "content": content
                }
                ]

    chat_completion = openai_client.chat.completions.create(
                        model=openai_model,
                        messages=messages,
                    )

    packed_answer = chat_completion.choices[0].message.content
    
    #return Markdown(packed_answer)
    return (packed_answer)


def format_markdown(content):        
    
    return Markdown(content)

In [ ]:
make_spr_by_file("./microsoft/transcripts/Microsoft Cloud in the era of AI - Scott Guthrie 2023.txt")

In [ ]:
import os
import pandas as pd

directory = "./microsoft/transcripts/"
chunk = {}
txt = []

for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), "r", encoding="utf-8") as f:
                text = f.read()
                texts = text_splitter.create_documents([text])
                doc_count = 0
                for i in texts:
                    chunk = {
                        "id": get_a_uuid(),  # generate a random uuid for the document
                        "title": f"{filename[:-4]}_Part_{doc_count}",  # remove the .txt extension from the filename and use this as the title
                        "content": i.page_content,
                        "spr": open_file("./microsoft/transcripts/spr/Scott_spr.txt"),
                        "sourcefile": filename,
                        "content_tokens": num_tokens_from_string(i.page_content, "gpt-4-1106-preview"),
                        "category": "Microsoft",
                        "contentVector": get_embedding(i.page_content)
                        }
                    txt.append(chunk)
                    doc_count += 1

df = pd.DataFrame(txt)
df

In [ ]:
df["title"]

In [ ]:
# populate a list with the data we will use to store in the index
def create_sections(df):
    for index, row in df.iterrows():
        yield {
            "id": row["id"],
            "title": row["title"],
            "content": row["content"],
            "spr": row["spr"],
            "sourcefile": row["sourcefile"],
            "category": row["category"],
            "contentVector": row["contentVector"],
            "@search.action": "upload",
        }
        
sections = create_sections(df)

In [ ]:
def index_sections(sections):
    print(
        f"Indexing sections into search index '{index_name}'"
    )

    search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

    i = 0
    batch = []
    for s in sections:
        batch.append(s)
        i += 1
        if i % 1000 == 0:
            results = search_client.upload_documents(documents=batch)
            succeeded = sum([1 for r in results if r.succeeded])
            print(f"\tIndexed {len(results)} sections, {succeeded} succeeded")
            batch = []

    if len(batch) > 0:
        results = search_client.upload_documents(documents=batch)
        succeeded = sum([1 for r in results if r.succeeded])
        print(f"\tIndexed {len(results)} sections, {succeeded} succeeded")
        
index_sections(sections)

In [ ]:
# Pure Vector Search
query = "helping people with disabilities"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=get_embedding(query), k_nearest_neighbors=5, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "category", "spr"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    #print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  
    #print(f"SPR: {result['spr']}\n") 

In [ ]:
# Hybrid
query = "bedrock"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
  
vector_query = VectorizedQuery(vector=get_embedding(query), k_nearest_neighbors=3, fields="contentVector")
  
category = "AWS"

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter=f"category eq '{category}'",
    #filter="category eq 'Microsoft'",
    select=["title", "content", "category", "spr"],
)
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  
    print(f"SPR: {result['spr']}\n")  